In [38]:
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [39]:
def load_asd_data_with_pandas(data_path: str) -> pd.DataFrame:
    """
    Загружает все parquet файлы из папки asd в один pandas DataFrame

    Args:
        data_path: путь к папке с данными

    Returns:
        pd.DataFrame: объединенный DataFrame со всеми данными
    """
    # Получаем все parquet файлы из папки
    parquet_files = glob.glob(os.path.join(data_path, "part-*.parquet"))

    if not parquet_files:
        raise ValueError(f"Не найдено parquet файлов в папке {data_path}")

    print(f"Найдено {len(parquet_files)} parquet файлов")

    # Загружаем все файлы в список DataFrame'ов
    dataframes = []
    for file_path in parquet_files:
        # print(f"Загружаем файл: {os.path.basename(file_path)}")
        df = pd.read_parquet(file_path)
        dataframes.append(df)

    # Объединяем все DataFrame'ы в один
    combined_df = pd.concat(dataframes, ignore_index=True)

    print(f"Общий размер данных: {combined_df.shape}")
    print(f"Колонки: {list(combined_df.columns)}")

    return combined_df

# Загружаем данные
agab_df = load_asd_data_with_pandas('./asd')

Найдено 20 parquet файлов
Общий размер данных: (1227083, 11)
Колонки: ['dataset', 'heavy_sequence', 'light_sequence', 'scfv', 'affinity_type', 'affinity', 'antigen_sequence', 'confidence', 'nanobody', 'metadata', 'processed_measurement']


### По файлам


In [ ]:
print('дубль — одинаковы все колонки, кроме metadata\n')
print("=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===\n")

parquet_files = glob.glob(os.path.join('./asd', "part-*.parquet"))
parquet_files.sort()

file_duplicates = []
for file_path in parquet_files:
    df = pd.read_parquet(file_path)
    file_name = os.path.basename(file_path)
    duplicates_in_file = df.drop(columns=['metadata']).duplicated().sum()
    file_duplicates.append({
        'file': file_name,
        'rows': len(df),
        'duplicates': duplicates_in_file,
        'duplicate_pct': duplicates_in_file / len(df) * 100 if len(df) > 0 else 0
    })

file_duplicates_df = pd.DataFrame(file_duplicates)
print("Дубликаты внутри каждого файла:")
print(file_duplicates_df.to_string(index=False))
print(f"\nОбщее количество дубликатов внутри файлов: {file_duplicates_df['duplicates'].sum()}")


=== АНАЛИЗ ДУБЛИКАТОВ ПО ФАЙЛАМ ===

Дубликаты внутри каждого файла:
                                                               file  rows  duplicates  duplicate_pct
part-00000-3a065afd-b2fa-4875-a6e5-911e95e3f86c-c000.snappy.parquet 61329        2777       4.528037
part-00001-74fdee0d-8448-4b0a-921c-f1f0ef356cdf-c000.snappy.parquet 60919        2740       4.497776
part-00002-634ededa-5adf-4170-93ba-2dac2bd74705-c000.snappy.parquet 63736        2550       4.000879
part-00003-427bc79e-0e40-4a8c-a2be-d0fbe09f03c0-c000.snappy.parquet 61152        2542       4.156855
part-00004-eb3dc336-995c-48bd-840f-49d411a89b8e-c000.snappy.parquet 60125        2936       4.883160
part-00005-846a5164-9ca8-4438-af5f-07ad5348f327-c000.snappy.parquet 61104        2816       4.608536
part-00006-b818506c-2926-406c-936b-66da5c9acdbc-c000.snappy.parquet 62158        3105       4.995334
part-00007-7ab8e466-84f8-43e0-8874-9c1bbf210d4e-c000.snappy.parquet 60256        2778       4.610329
part-00008-397aa529-5c

In [16]:
print("\n=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===\n")

from itertools import combinations

file_unique_sets = {os.path.basename(fp): set(pd.read_parquet(fp).drop(columns=['metadata']).apply(tuple, axis=1)) for fp in parquet_files}

overlaps = [{'file1': f1, 'file2': f2, 'overlap_count': overlap, 'file1_size': len(s1), 'file2_size': len(s2), 
             'overlap_pct_file1': overlap / len(s1) * 100 if len(s1) > 0 else 0, 'overlap_pct_file2': overlap / len(s2) * 100 if len(s2) > 0 else 0}
            for (f1, s1), (f2, s2) in combinations(file_unique_sets.items(), 2) if (overlap := len(s1 & s2)) > 0]

if overlaps:
    overlaps_df = pd.DataFrame(overlaps)
    print("Найдены перекрытия между файлами:")
    print(overlaps_df.to_string(index=False))
    print(f"\nВсего пар файлов с перекрытиями: {len(overlaps)}")
    print(f"Общее количество перекрывающихся записей: {overlaps_df['overlap_count'].sum()}")
else:
    print("Перекрытий между файлами не обнаружено.")



=== АНАЛИЗ ПЕРЕКРЫТИЙ МЕЖДУ ФАЙЛАМИ ===

Перекрытий между файлами не обнаружено.


### По датасетам

In [40]:
print('дубль — одинаковы все колонки, кроме metadata\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

dataset_analysis = []
for dataset in agab_df['dataset'].unique():
    dataset_df = agab_df[agab_df['dataset'] == dataset]
    duplicates = dataset_df.drop(columns=['metadata']).duplicated().sum()
    dataset_analysis.append({
        'dataset': dataset,
        'total_rows': len(dataset_df),
        'duplicates': duplicates,
        'duplicate_pct': duplicates / len(dataset_df) * 100 if len(dataset_df) > 0 else 0,
        'unique_rows': len(dataset_df) - duplicates
    })

dataset_analysis_df = pd.DataFrame(dataset_analysis).sort_values('duplicates', ascending=False)
print("Дубликаты по датасетам:")
print(dataset_analysis_df.to_string(index=False))


дубль — одинаковы все колонки, кроме metadata


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===

Дубликаты по датасетам:
                dataset  total_rows  duplicates  duplicate_pct  unique_rows
               covid-19       54625       27482      50.310297        27143
                    hiv       48008       24715      51.481003        23293
                 biomap        2725        1402      51.449541         1323
  structures-antibodies        2711        1170      43.157506         1541
                genbank        2989         717      23.987956         2272
  structures-nanobodies        1258         649      51.589825          609
                patents      217463         164       0.075415       217299
                   abbd      155853         132       0.084695       155721
               alphaseq      198703           4       0.002013       198699
flab_shanehsazzadeh2023         446           1       0.224215          445
               abdesign         672           0     

In [43]:
print('дубль — одинаковы все колонки\n')
print("\n=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===\n")

# Добавляем флаг дубликатов в исходный DataFrame
agab_df['is_duplicate'] = duplicates_with_metadata_mask

# Группируем по dataset и считаем статистику
dataset_duplicates = agab_df.groupby('dataset').agg({
    'is_duplicate': ['count', 'sum', 'mean']
}).round(4)

# Переименовываем столбцы для удобства
dataset_duplicates.columns = ['total_rows', 'duplicates_count', 'duplicates_ratio']
dataset_duplicates = dataset_duplicates.reset_index()

# Сортируем по количеству дубликатов
dataset_duplicates = dataset_duplicates.sort_values('duplicates_count', ascending=False)

print("\nТопdataset по количеству дубликатов:")
print(dataset_duplicates.to_string(index=False))

# Общая статистика по распределению
print(f"\nВсего dataset: {len(dataset_duplicates)}")
print(f"Dataset с дубликатами: {len(dataset_duplicates[dataset_duplicates['duplicates_count'] > 0])}")
print(f"Dataset без дубликатов: {len(dataset_duplicates[dataset_duplicates['duplicates_count'] == 0])}")

дубль — одинаковы все колонки


=== АНАЛИЗ ДУБЛИКАТОВ ПО ДАТАСЕТАМ ===


Топdataset по количеству дубликатов:
                dataset  total_rows  duplicates_count  duplicates_ratio
                    hiv       48008             24715            0.5148
                 biomap        2725              1402            0.5145
  structures-antibodies        2711               999            0.3685
  structures-nanobodies        1258               566            0.4499
                genbank        2989               480            0.1606
               covid-19       54625               303            0.0055
                   abbd      155853               132            0.0008
               alphaseq      198703                 4            0.0000
flab_shanehsazzadeh2023         446                 1            0.0022
               skempiv2         434                 0            0.0000
                   rmna          10                 0            0.0000
                patents   

### Cтатистика

In [3]:
print("=== Статистика дубликатов ===")
print(f"Всего строк: {len(agab_df)}")

# Создаем маску дубликатов один раз для эффективности
duplicates_mask = agab_df.drop(columns='metadata').duplicated()
duplicates_count = duplicates_mask.sum()

print(f"Дубликатов: {duplicates_count}")
print(f"Уникальных строк: {len(agab_df) - duplicates_count}")


print("\n=== Дубликаты с учетом metadata ===")

# Создаем временный DataFrame, преобразуя metadata в строку JSON
temp_df = agab_df.copy()
temp_df['metadata_json'] = temp_df['metadata'].apply(lambda x: json.dumps(x, sort_keys=True) if isinstance(x, dict) else str(x))

# Проверяем дубликаты, исключая столбец metadata (используем все столбцы кроме metadata)
columns_to_check = [col for col in temp_df.columns if col != 'metadata']
duplicates_with_metadata_mask = temp_df[columns_to_check].duplicated()
duplicates_with_metadata_count = duplicates_with_metadata_mask.sum()

print(f"Дубликатов с учетом metadata: {duplicates_with_metadata_count}")
print(f"Уникальных строк с учетом metadata: {len(agab_df) - duplicates_with_metadata_count}")

print("\n=== Сравнительная статистика ===")
print(f"Разница в количестве дубликатов: {duplicates_count - duplicates_with_metadata_count}")

=== Статистика дубликатов ===
Всего строк: 1227083
Дубликатов: 56436
Уникальных строк: 1170647

=== Дубликаты с учетом metadata ===
Дубликатов с учетом metadata: 28602
Уникальных строк с учетом metadata: 1198481

=== Сравнительная статистика ===
Разница в количестве дубликатов: 27834


### Различия metadata в дублях

In [ ]:
# на примере covid-19
covid_df = agab_df[
    (agab_df['dataset'] == 'covid-19')
    & (agab_df['scfv'] == False)
    & (agab_df['nanobody'] == False)
]

In [35]:
from collections import defaultdict, Counter


print("=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В COVID-19 ===\n")

# Находим дубликаты в covid_df по всем колонкам кроме metadata
covid_duplicates_mask = covid_df.drop(columns=['metadata']).duplicated(keep=False)
covid_duplicates = covid_df[covid_duplicates_mask].copy()

# Создаем ключ для группировки
covid_duplicates['duplicate_key'] = covid_duplicates.drop(columns=['metadata']).apply(tuple, axis=1)

# Получаем уникальные группы дубликатов
unique_duplicate_groups = covid_duplicates['duplicate_key'].unique()

print(f"Всего групп дубликатов: {len(unique_duplicate_groups)}\n")

# Функция для рекурсивного сравнения словарей и поиска различий
def find_different_keys(dict1, dict2, path=""):
    """Находит все ключи, которые различаются между двумя словарями"""
    different_keys = set()
    
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        if dict1 != dict2:
            return {path} if path else set()
        return set()
    
    all_keys = set(dict1.keys()) | set(dict2.keys())
    
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key
        val1 = dict1.get(key)
        val2 = dict2.get(key)
        
        if key not in dict1:
            different_keys.add(current_path)
        elif key not in dict2:
            different_keys.add(current_path)
        elif isinstance(val1, dict) and isinstance(val2, dict):
            different_keys.update(find_different_keys(val1, val2, current_path))
        elif val1 != val2:
            different_keys.add(current_path)
    
    return different_keys

# Собираем статистику по различиям
field_differences = Counter()
groups_with_differences = 0
groups_with_same_metadata = 0
total_groups_analyzed = 0

# Анализируем каждую группу дубликатов
for group_key in unique_duplicate_groups:
    group_data = covid_duplicates[covid_duplicates['duplicate_key'] == group_key]
    
    if len(group_data) < 2:
        continue
    
    total_groups_analyzed += 1
    
    # Сравниваем metadata между всеми парами в группе
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, все ли metadata одинаковые
    all_same = True
    group_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                all_same = False
                group_differences.update(diff_keys)
    
    if all_same:
        groups_with_same_metadata += 1
    else:
        groups_with_differences += 1
        for key in group_differences:
            field_differences[key] += 1

print(f"Проанализировано групп: {total_groups_analyzed}")
print(f"Групп с одинаковым metadata: {groups_with_same_metadata} ({groups_with_same_metadata/total_groups_analyzed*100:.2f}%)")
print(f"Групп с различиями в metadata: {groups_with_differences} ({groups_with_differences/total_groups_analyzed*100:.2f}%)\n")

if field_differences:
    print("=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_differences:
        pct = count / groups_with_differences * 100
        print(f"{field}: {count} групп ({pct:.2f}% от групп с различиями)")
    
    print(f"\nВсего уникальных полей с различиями: {len(field_differences)}")
else:
    print("Различий в metadata между дубликатами не найдено.")

# Показываем примеры групп с различиями
print("\n=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===\n")

examples_shown = 0
for group_key in unique_duplicate_groups:
    if examples_shown >= 5:
        break
    
    group_data = covid_duplicates[covid_duplicates['duplicate_key'] == group_key].copy()
    
    if len(group_data) < 2:
        continue
    
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, есть ли различия
    has_differences = False
    all_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                has_differences = True
                all_differences.update(diff_keys)
    
    if has_differences:
        examples_shown += 1
        print(f"{'='*80}")
        print(f"Пример {examples_shown}: Группа из {len(group_data)} дубликатов")
        print(f"Различающиеся поля: {', '.join(sorted(all_differences))}")
        print(f"{'='*80}\n")
        
        # Показываем первые 2 записи из группы
        for idx, (row_idx, row) in enumerate(group_data.head(2).iterrows()):
            print(f"--- Запись {idx+1} (индекс {row_idx}) ---")
            metadata_str = json.dumps(row['metadata'], indent=2, ensure_ascii=False) if isinstance(row['metadata'], dict) else str(row['metadata'])
            print(f"metadata:\n{metadata_str}\n")
        
        print()

# Удаляем временную колонку
covid_duplicates.drop(columns=['duplicate_key'], inplace=True)


=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В COVID-19 ===

Всего групп дубликатов: 27142

Проанализировано групп: 27142
Групп с одинаковым metadata: 0 (0.00%)
Групп с различиями в metadata: 27142 (100.00%)

=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===



ValueError: too many values to unpack (expected 2)

In [37]:
# они друг за другом идут в covid-19
for idx, meta in covid_df.iloc[1000:1002]['metadata'].items():
    print(f'Индекс {idx}:\n', json.dumps(meta, ensure_ascii=False, indent=2), '\n')

Индекс 55265:
 {
  "target_name": null,
  "target_pdb": "",
  "target_uniprot": "",
  "source_url": "https://academic.oup.com/bib/article/26/1/bbaf008/7964432",
  "heavy_riot_numbering": {
    "cdr1_aa": "GFTFTSSA",
    "cdr2_aa": "IVVGSGNT",
    "cdr3_aa": "AAPYCSGGSCFDGFDI",
    "sequence_alignment_aa": "QMQLVQSGPEVKKPGTSVKVSCKASGFTFTSSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAPYCSGGSCFDGFDIWGQGTMVTVSS"
  },
  "light_riot_numbering": {
    "cdr1_aa": null,
    "cdr2_aa": null,
    "cdr3_aa": null,
    "sequence_alignment_aa": null
  }
} 

Индекс 55266:
 {
  "target_name": "sars-cov2_omicron",
  "target_pdb": "",
  "target_uniprot": "",
  "source_url": "https://academic.oup.com/bioinformatics/article/37/5/734/5893556",
  "heavy_riot_numbering": {
    "cdr1_aa": "GFTFTSSA",
    "cdr2_aa": "IVVGSGNT",
    "cdr3_aa": "AAPYCSGGSCFDGFDI",
    "sequence_alignment_aa": "QMQLVQSGPEVKKPGTSVKVSCKASGFTFTSSAVQWVRQARGQRLEWIGWIVVGSGNTNYAQKFQERVTITRDMSTSTAYMELSSLRSEDTAVYYCAAP

In [34]:
print("=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В AGAB ===\n")

# Находим дубликаты в agab_df по всем колонкам кроме metadata
agab_duplicates_mask = agab_df.drop(columns=['metadata']).duplicated(keep=False)
agab_duplicates = agab_df[agab_duplicates_mask].copy()

# Создаем ключ для группировки
agab_duplicates['duplicate_key'] = agab_duplicates.drop(columns=['metadata']).apply(tuple, axis=1)

# Получаем уникальные группы дубликатов
unique_duplicate_groups = agab_duplicates['duplicate_key'].unique()

print(f"Всего групп дубликатов: {len(unique_duplicate_groups)}\n")

# Функция для рекурсивного сравнения словарей и поиска различий
def find_different_keys(dict1, dict2, path=""):
    """Находит все ключи, которые различаются между двумя словарями"""
    different_keys = set()
    
    if not isinstance(dict1, dict) or not isinstance(dict2, dict):
        if dict1 != dict2:
            return {path} if path else set()
        return set()
    
    all_keys = set(dict1.keys()) | set(dict2.keys())
    
    for key in all_keys:
        current_path = f"{path}.{key}" if path else key
        val1 = dict1.get(key)
        val2 = dict2.get(key)
        
        if key not in dict1:
            different_keys.add(current_path)
        elif key not in dict2:
            different_keys.add(current_path)
        elif isinstance(val1, dict) and isinstance(val2, dict):
            different_keys.update(find_different_keys(val1, val2, current_path))
        elif val1 != val2:
            different_keys.add(current_path)
    
    return different_keys

# Собираем статистику по различиям
field_differences = Counter()
groups_with_differences = 0
groups_with_same_metadata = 0
total_groups_analyzed = 0

# Анализируем каждую группу дубликатов
for group_key in unique_duplicate_groups:
    group_data = agab_duplicates[agab_duplicates['duplicate_key'] == group_key]
    
    if len(group_data) < 2:
        continue
    
    total_groups_analyzed += 1
    
    # Сравниваем metadata между всеми парами в группе
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, все ли metadata одинаковые
    all_same = True
    group_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                all_same = False
                group_differences.update(diff_keys)
    
    if all_same:
        groups_with_same_metadata += 1
    else:
        groups_with_differences += 1
        for key in group_differences:
            field_differences[key] += 1

print(f"Проанализировано групп: {total_groups_analyzed}")
print(f"Групп с одинаковым metadata: {groups_with_same_metadata} ({groups_with_same_metadata/total_groups_analyzed*100:.2f}%)")
print(f"Групп с различиями в metadata: {groups_with_differences} ({groups_with_differences/total_groups_analyzed*100:.2f}%)\n")

if field_differences:
    print("=== ПОЛЯ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===\n")
    for field, count in field_differences:
        pct = count / groups_with_differences * 100
        print(f"{field}: {count} групп ({pct:.2f}% от групп с различиями)")
    
    print(f"\nВсего уникальных полей с различиями: {len(field_differences)}")
else:
    print("Различий в metadata между дубликатами не найдено.")

# Показываем примеры групп с различиями
print("\n=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===\n")

examples_shown = 0
for group_key in unique_duplicate_groups:
    if examples_shown >= 5:
        break
    
    group_data = agab_duplicates[agab_duplicates['duplicate_key'] == group_key].copy()
    
    if len(group_data) < 2:
        continue
    
    metadata_list = group_data['metadata'].tolist()
    
    # Проверяем, есть ли различия
    has_differences = False
    all_differences = set()
    
    for i in range(len(metadata_list)):
        for j in range(i + 1, len(metadata_list)):
            meta1 = metadata_list[i] if isinstance(metadata_list[i], dict) else {}
            meta2 = metadata_list[j] if isinstance(metadata_list[j], dict) else {}
            
            diff_keys = find_different_keys(meta1, meta2)
            if diff_keys:
                has_differences = True
                all_differences.update(diff_keys)
    
    if has_differences:
        examples_shown += 1
        print(f"{'='*80}")
        print(f"Пример {examples_shown}: Группа из {len(group_data)} дубликатов")
        print(f"Различающиеся поля: {', '.join(sorted(all_differences))}")
        print(f"{'='*80}\n")
        
        # Показываем первые 2 записи из группы
        for idx, (row_idx, row) in enumerate(group_data.head(2).iterrows()):
            print(f"--- Запись {idx+1} (индекс {row_idx}) ---")
            
            # Показываем все колонки кроме duplicate_key
            display_cols = [col for col in group_data.columns if col not in ['duplicate_key']]
            
            for col in display_cols:
                if col == 'metadata':
                    # Красиво выводим metadata
                    metadata_str = json.dumps(row[col], indent=2, ensure_ascii=False) if isinstance(row[col], dict) else str(row[col])
                    print(f"{col}:\n{metadata_str}")
                else:
                    print(f"{col}: {row[col]}")
            
            print()
        
        print()

# Удаляем временную колонку
agab_duplicates.drop(columns=['duplicate_key'], inplace=True)


=== АНАЛИЗ РАЗЛИЧИЙ В METADATA МЕЖДУ ДУБЛИКАТАМИ В AGAB ===

Всего групп дубликатов: 53478

Проанализировано групп: 53478
Групп с одинаковым metadata: 25872 (48.38%)
Групп с различиями в metadata: 27606 (51.62%)

=== ТОП-20 ПОЛЕЙ METADATA, КОТОРЫЕ ЧАЩЕ ВСЕГО РАЗЛИЧАЮТСЯ ===

target_name: 27433 групп (99.37% от групп с различиями)
source_url: 27142 групп (98.32% от групп с различиями)
target_uniprot: 225 групп (0.82% от групп с различиями)
target_pdb: 133 групп (0.48% от групп с различиями)

Всего уникальных полей с различиями: 4

=== ПРИМЕРЫ ГРУПП С РАЗЛИЧИЯМИ В METADATA ===

Пример 1: Группа из 4 дубликатов
Различающиеся поля: target_pdb

--- Запись 1 (индекс 33) ---
dataset: structures-nanobodies
heavy_sequence: EVQLLESGGGLVQPGGSLRLSCAASGFRFDAEDMGWVRQAPGKGLEWVSSIYGPSGSTYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCAKYTSPPQNHGFDYWGQGTLVTVSS
light_sequence: 
scfv: False
affinity_type: bool
affinity: 1.0
antigen_sequence: KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSR